<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/MIMIC/mimic_bregman_classification_modded_fullseq_length.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers transformers datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset
import json

In [ ]:
!git clone https://github.com/danielsaggau/IR_LDC.git

In [ ]:
!unzip /content/drive/MyDrive/mimic.jsonl.zip -d content

In [10]:
import shutil
shutil.copy("/content/content/mimic.jsonl", "/content/content/IR_LDC/model/MIMIC")
dataset = load_dataset("/content/content/IR_LDC/model/MIMIC/mimic-dataset.py")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset mimic-dataset downloaded and prepared to /root/.cache/huggingface/datasets/mimic-dataset/mimic/1.1.0/90aef5b28cbcdff1c522e7dd5b743336977fe255361a97e4c5035fbd7da6e512. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
dataset_train_test = dataset['train'].train_test_split(test_size=0.1)
dataset_test = dataset_train_test['test']
dataset_sp = dataset_train_test['train'].train_test_split(test_size=0.1/0.9)
dataset_train = dataset_sp['train']
dataset_validation = dataset_sp['test']

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXX')"
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/simcse_mimic_biobert_long/10000", use_auth_token=True, use_fast=True)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/simcse_mimic_biobert_long/10000", use_auth_token=True,num_labels=19, problem_type='multi_label_classification')

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/simcse_mimic_biobert_long/10000 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) # fp16

In [16]:
train_dataset=dataset['train']

In [17]:
num_labels = train_dataset.features['labels'].feature.num_classes
label_ids = train_dataset.features['labels'].feature.names

label_names = label_ids
label_list = list(range(num_labels))

In [18]:
   def preprocess_function(examples):
        # Tokenize the texts
        batch = tokenizer(
            examples["text"],
            padding='max_length',
            max_length=2048, # for full run 4096
            truncation=True)
        
        batch = tokenizer.pad(
            batch,
            padding='max_length',
            max_length=2048, # for full run 4096
            pad_to_multiple_of=8,
        )
        batch["label_ids"] = [[1.0 if label in labels else 0.0 for label in label_list] for labels in examples["labels"]]
        return batch

In [19]:
tokenized_data = dataset.map(preprocess_function, batched=True, remove_columns=['labels'])

  0%|          | 0/30 [00:00<?, ?ba/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [20]:
   from transformers import EvalPrediction
   def compute_metrics(p: EvalPrediction):
        logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = (expit(logits) > 0.5).astype(int)
        label_ids = (p.label_ids > 0.5).astype(int)
        macro_f1 = f1_score(y_true=label_ids, y_pred=preds, average='macro', zero_division=0)
        micro_f1 = f1_score(y_true=label_ids, y_pred=preds, average='micro', zero_division=0)
        return {'macro_f1': macro_f1, 'micro_f1': micro_f1}

In [21]:
from transformers import TrainingArguments, EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="modded_mimic_frozen_simcse",
    learning_rate=1e-3,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=9,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,
    push_to_hub=True,
    metric_for_best_model="micro_f1",
    greater_is_better=True,
    load_best_model_at_end = True,
    report_to="wandb",
    run_name="mimic_modded_frozen_simcse")

In [ ]:
tokenized_data['test']

In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
#Bert pooling
import torch
from torch import nn
class BertMeanPooler(nn.Module):
          def __init__(self, config):
             super().__init__()
             self.dense = nn.Linear(config.hidden_size, config.hidden_size)
             self.activation = nn.Tanh()

          def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
              mean_token_tensor = hidden_states.mean(dim=1)
              pooled_output = self.dense(mean_token_tensor)
              pooled_output = self.activation(pooled_output)
              return pooled_output
model.longformer.pooler = BertMeanPooler(model.config)
print('model mean pooler loaded')

model mean pooler loaded


In [25]:
for param in model.longformer.parameters():
    param.requires_grad = False

In [ ]:
# !wandb login XXXX

In [27]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 10.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=13e03340ef0d38aefcb4200633888f0414b73d14268f2a6c935bb6f41d0d2ed8
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [ ]:
from transformers import Trainer
from scipy.special import expit
from sklearn.metrics import f1_score
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    eval_dataset=tokenized_data['test'],
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)])
trainer.train()

Initializing global attention on CLS token...


Epoch,Training Loss,Validation Loss,Macro F1,Micro F1
1,0.443400,0.436622,0.457411,0.627425
2,0.439600,0.427865,0.455287,0.609455
3,0.434300,0.422852,0.488230,0.641183
4,0.437000,0.421807,0.514501,0.635486
5,0.426400,0.417831,0.531448,0.660608
6,0.427700,0.415453,0.546202,0.651731
7,0.421300,0.411551,0.544301,0.651501


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing 

In [ ]:
trainer.evaluate(eval_dataset=tokenized_data['validation'])

In [ ]:
from google.colab import runtime
runtime.unassign()